In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('rajasthan_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while accessing page 3: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF748979412+29090]
	(No symbol) [0x00007FF7488EE239]
	(No symbol) [0x00007FF7487AB1DA]
	(No symbol) [0x00007FF7487FEFE7]
	(No symbol) [0x00007FF7487FF23C]
	(No symbol) [0x00007FF7488497C7]
	(No symbol) [0x00007FF74882672F]
	(No symbol) [0x00007FF7488465A2]
	(No symbol) [0x00007FF748826493]
	(No symbol) [0x00007FF7487F09D1]
	(No symbol) [0x00007FF7487F1B31]
	GetHandleVerifier [0x00007FF748C9871D+3302573]
	GetHandleVerifier [0x00007FF748CE4243+3612627]
	GetHandleVerifier [0x00007FF748CDA417+3572135]
	GetHandleVerifier [0x00007FF748A35EB6+801862]
	(No symbol) [0x00007FF7488F945F]
	(No symbol) [0x00007FF7488F4FB4]
	(No symbol) [0x00007FF7488F5140]
	(No symbol) [0x00007FF7488E461F]
	BaseThreadInitThunk [0x00007FF9AB07257D+29]
	RtlUserThreadStart [0x00007FF9ABC6AF28+40]



NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 188347,Super Luxury Volvo AC Seater Pushback 2+2,11:30,05h 00m,16:30,4.2,INR 602,2 Seats available
1,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 190115,Express Non AC Seater 2+3,12:00,07h 30m,19:30,4.2,INR 294,41 Seats available
2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 169076,Express Non AC Seater 2+3,12:10,11h 50m,00:00,4.2,INR 299,42 Seats available
3,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 184043,Express Non AC Seater 2+3,12:45,06h 15m,19:00,4.2,INR 299,42 Seats available
4,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...,RSRTC - 183203,Express Non AC Seater 2+3,13:30,06h 00m,19:30,4.2,INR 299,42 Seats available
...,...,...,...,...,...,...,...,...,...,...
849,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Chouhan travels and Cargo,NON A/C Sleeper (2+1),22:00,05h 05m,03:05,3.3,INR 700,36 Seats available
850,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Hans Travels (I) Private Limited,AC Sleeper (2+1),20:30,05h 15m,01:45,3.2,INR 1000,36 Seats available
851,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Chouhan Travels,NON A/C Sleeper (2+1),20:15,03h 35m,23:50,3.0,INR 596,24 Seats available
852,Jaipur (Rajasthan) to Kota(Rajasthan),https://www.redbus.in/bus-tickets/jaipur-to-ko...,Navrang Sanjay Travels,AC Sleeper (2+1),23:00,06h 00m,05:00,2.9,INR 500,34 Seats available
